In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import ipywidgets as widgets
from IPython.display import display
import matplotlib

# Use TkAgg backend for dynamic plots
matplotlib.use('TkAgg')

class SchellingSimulation:
    def __init__(self, grid_size=50, num_agents_A=600, num_agents_B=600, similar_neighbors_A=3, similar_neighbors_B=3):
        self.grid_size = grid_size
        self.num_agents_A = num_agents_A
        self.num_agents_B = num_agents_B
        self.similar_neighbors_A = similar_neighbors_A
        self.similar_neighbors_B = similar_neighbors_B
        self.grid = np.zeros((grid_size, grid_size), dtype=int)
        self.initialize_grid()
        self.fig, self.ax = plt.subplots()
        self.cmap = plt.cm.colors.ListedColormap(['white', 'blue', 'red'])
        self.bounds = [0, 1, 2, 3]
        self.norm = plt.cm.colors.BoundaryNorm(self.bounds, self.cmap.N)
        self.im = self.ax.imshow(self.grid, cmap=self.cmap, norm=self.norm)

    def initialize_grid(self):
        self.grid = np.zeros((self.grid_size, self.grid_size), dtype=int)
        positions_A = np.random.choice(self.grid_size * self.grid_size, self.num_agents_A, replace=False)
        positions_B = np.random.choice(self.grid_size * self.grid_size, self.num_agents_B, replace=False)

        for pos in positions_A:
            self.grid[pos // self.grid_size, pos % self.grid_size] = 1  # Agent A

        for pos in positions_B:
            self.grid[pos // self.grid_size, pos % self.grid_size] = 2  # Agent B

    def count_similar_neighbors(self, x, y, agent_type):
        directions = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]
        similar_count = 0
        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if 0 <= nx < self.grid_size and 0 <= ny < self.grid_size:
                if self.grid[nx, ny] == agent_type:
                    similar_count += 1
        return similar_count

    def move_agent(self, x, y, agent_type):
        empty_positions = np.argwhere(self.grid == 0)
        if len(empty_positions) == 0:
            return False
        new_x, new_y = empty_positions[np.random.choice(len(empty_positions))]
        self.grid[x, y] = 0
        self.grid[new_x, new_y] = agent_type
        return True

    def update(self, frame):
        changed = False
        for x in range(self.grid_size):
            for y in range(self.grid_size):
                agent_type = self.grid[x, y]
                if agent_type == 1:
                    if self.count_similar_neighbors(x, y, 1) < self.similar_neighbors_A:
                        if self.move_agent(x, y, 1):
                            changed = True
                elif agent_type == 2:
                    if self.count_similar_neighbors(x, y, 2) < self.similar_neighbors_B:
                        if self.move_agent(x, y, 2):
                            changed = True
        if not changed:
            self.ani.event_source.stop()
        self.im.set_array(self.grid)
        return [self.im]

    def run(self):
        self.ani = animation.FuncAnimation(self.fig, self.update, frames=100, interval=200, repeat=False)
        plt.show()

# Widget controls
agent_A_slider = widgets.IntSlider(value=600, min=0, max=2500, step=1, description='Agents A')
agent_B_slider = widgets.IntSlider(value=600, min=0, max=2500, step=1, description='Agents B')
similar_A_slider = widgets.IntSlider(value=3, min=0, max=8, step=1, description='Similar A')
similar_B_slider = widgets.IntSlider(value=3, min=0, max=8, step=1, description='Similar B')
run_button = widgets.Button(description='Run Simulation')

# Interactive output
def run_simulation(_):
    sim = SchellingSimulation(
        num_agents_A=agent_A_slider.value,
        num_agents_B=agent_B_slider.value,
        similar_neighbors_A=similar_A_slider.value,
        similar_neighbors_B=similar_B_slider.value
    )
    sim.run()

run_button.on_click(run_simulation)

# Display controls
display(agent_A_slider, agent_B_slider, similar_A_slider, similar_B_slider, run_button)

IntSlider(value=600, description='Agents A', max=2500)

IntSlider(value=600, description='Agents B', max=2500)

IntSlider(value=3, description='Similar A', max=8)

IntSlider(value=3, description='Similar B', max=8)

Button(description='Run Simulation', style=ButtonStyle())